In [ ]:
from PyBayesAB import bernoulli, poisson, normal, multinomial
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Bernoulli 

In [ ]:

# Create data type = Bernoulli object
Bern_test = bernoulli.BaysBernoulli()

# create data for two groups
p_A = 0.21
p_B = 0.2
for n in range(20):
    n_trial = np.random.randint(10,50)
    Bern_test.add_rand_experiment(n_trial, p_A, group="A")
    Bern_test.add_rand_experiment(n_trial, p_B, group="B")


In [ ]:
# calculate some Bayesian metrics

ROPE = [-0.01, 0.01]  # Region of Practical Equivalence

print(Bern_test.summary_result(rope_interval=ROPE,level=95))

_, _ = Bern_test.plot_bayesian_metrics(rope_interval=ROPE)

In [ ]:
# plot posterior of the difference in p
Bern_test.plot_final_posterior(group="AB")
plt.show()

In [ ]:
# plot cumulative (updated) posterior of the difference in p
Bern_test.plot_cum_posterior(group="A", type="1D")
plt.show()

In [ ]:
# Plot cumulative (updated) posterior for both groups as heatmap
Bern_test.plot_cum_posterior(group="AB", type="2D")
plt.show()

In [ ]:
# Plot cumulative (updated) posterior for the difference as heatmap
Bern_test.plot_cum_posterior(group="diff", type="2D")
plt.show()

In [ ]:
Bern_test.plot_cum_posterior(group="AB", type="3D")
plt.show()

In [ ]:
# Plot animation for both groups
anim = Bern_test.plot_anim(group="AB")
anim

In [ ]:
# return loss-based decision
Bern_test.loss_based_decision(loss_weight=2.)

# Poisson

In [ ]:
from PyBayesAB.distribution import poisson

# Create Likelihhod = Poisson object
Pois_test = poisson.BaysPoisson()

# create data
mu_A = 20
mu_B = 21
n_exp = 20
for next in range(n_exp):
    n_events = np.random.randint(5, 10)
    # add random experiment for group A and B
    Pois_test.add_rand_experiment(n_events, mu_A)
    Pois_test.add_rand_experiment(n_events, mu_B, group="B")

In [ ]:
# calculate some Bayesian metrics

ROPE = [-2, 2]  # Region of Practical Equivalence

print(Pois_test.summary_result(rope_interval=ROPE,level=95))

_, _ = Pois_test.plot_bayesian_metrics(rope_interval=ROPE)

In [ ]:
# Final posterior difference in Poisson mean
Pois_test.plot_final_posterior(group="AB")
plt.show()


In [ ]:
Pois_test.plot_cum_posterior(group="AB", type="2D")
plt.show()


In [ ]:
Pois_test.plot_cum_posterior(group="AB", type="3D", para_range=[15, 25])
plt.show()

In [ ]:
# animation
anim = Pois_test.plot_anim(group="B",  para_range=[10,25])
anim

# Normal

## Normal  with unkown mean and std

In [ ]:
from PyBayesAB.distribution import normal
import numpy as np

mu_A = 20
std_A = 10
tau_A = 1/std_A**2 
mu_B = 22
std_B = 12
tau_B = 1/std_B**2
normal = normal.BaysNorm()
n_exp = 20
for i in range(n_exp):
    n_data = np.random.randint(10,50)
    normal.add_rand_experiment(n_data, mu_A, std_A, group="A")
    normal.add_rand_experiment(n_data, mu_B, std_B, group="B")

In [ ]:
from scipy.stats import multinomial as scipy_mult
scipy_mult.rvs(n=10, p=[0.2, 0.3, 0.5], size=1)

In [ ]:
normal.plot_final_posterior(group="AB")
plt.show()

In [ ]:
normal.plot_final_posterior(group="diff", post_kwargs={"var":"std"})
plt.show()

In [ ]:
# calculate some Bayesian metrics

ROPE = [-1, 1]  # Region of Practical Equivalence
print(normal.summary_result(rope_interval=ROPE,level=95))

normal.plot_bayesian_metrics(rope_interval=ROPE)

In [ ]:
normal.plot_cum_posterior(group="diff", type="3D",  para_range=[-10,10])
plt.show()

In [ ]:
# animation
anim = normal.plot_anim(group="diff", para_range=[-10,10])
anim

# Multinomial

In [ ]:
# generate  random multinomial data

# Create data type = Bernoulli object
Multi_test = multinomial.BaysMultinomial(prior=np.ones(3))

# create data for two groups
p_A = [0.1, 0.2,  0.7]
p_B = [0.2, 0.5, 0.3]
for n in range(20):
    n_trial = np.random.randint(10,50)
    Multi_test.add_rand_experiment(n_trial, p_A, group="A")
    Multi_test.add_rand_experiment(n_trial, p_B, group="B")

In [ ]:
Multi_test.plot_dirichlet_rvs(group="diff")
plt.show()

In [ ]:
Multi_test.plot_cum_posterior(category_idx=2, group="diff", type="3D")
plt.show()


In [ ]:
# Difference — ribbons around zero showing A-B per category
fig = Multi_test.plot_cum_posterior_dirichlet(group="diff")
plt.show()

# Custom CI level
fig = Multi_test.plot_cum_posterior_dirichlet(group="diff", level=90)
plt.show()


# Composite model

In [ ]:
# Poisson * Normal model

n_exp = 20

poisson_model = poisson.BaysPoisson()
mp_A = 20
mp_B = 15
for n in range(n_exp):
    n_trial = np.random.randint(10,50)
    poisson_model.add_rand_experiment(n_trial, mp_A, group="A")
    poisson_model.add_rand_experiment(n_trial, mp_B, group="B")

normal_model = normal.BaysNorm()
mu_A = 20
std_A = 10
tau_A = 1/std_A**2 
mu_B = 22
std_B = 12
tau_B = 1/std_B**2
for i in range(n_exp):
    n_data = np.random.randint(10,50)
    normal_model.add_rand_experiment(n_data, mu_A, std_A, group="A")
    normal_model.add_rand_experiment(n_data, mu_B, std_B, group="B")

truth =  (mp_A*mu_A)-(mp_B*mu_B)

# create a composite model
composite_model = poisson_model * normal_model

# plot the final posterior for the composite model
fig = composite_model.plot_final_posterior()
plt.axvline(truth,color="black")
plt.text(x=truth, y=0.03, s=f"Truth = {truth}", color="black")
plt.show()

# get Bayesian statistics for the composite model
stats = composite_model.summary_result(rope_interval=[-5,  5])
print(stats)
print("True target: ", (mp_A*mu_A)-(mp_B*mu_B))

composite_model.plot_bayesian_metrics(rope_interval=[-20, 20])
plt.show()

In [ ]:
# Customer purchase model

bern = bernoulli.BaysBernoulli()
multi = multinomial.BaysMultinomial(prior=np.ones(3))
gauss0 = normal.BaysNorm()
gauss1 = normal.BaysNorm()
gauss2 = normal.BaysNorm()

# 15 experiments
for i in range(15):
    bern.add_experiment(np.random.binomial(1, 0.7), group='A')
    bern.add_experiment(np.random.binomial(1, 0.6), group='B')
    multi.add_rand_experiment(100, [0.3, 0.5, 0.2], group='A')
    multi.add_rand_experiment(100, [0.4, 0.4, 0.2], group='B')
    gauss0.add_experiment(np.random.normal(20, 3), group='A')
    gauss0.add_experiment(np.random.normal(18, 3), group='B')
    gauss1.add_experiment(np.random.normal(50, 5), group='A')
    gauss1.add_experiment(np.random.normal(45, 5), group='B')
    gauss2.add_experiment(np.random.normal(100, 10), group='A')
    gauss2.add_experiment(np.random.normal(90, 10), group='B')

# Full customer model: expected revenue = p_buy * sum(p_product_i * price_i)
revenue_model = bern * (multi[0]*gauss0 + multi[1]*gauss1 + multi[2]*gauss2)
print(f'Model type: {type(revenue_model).__name__}')

# Posterior samples
rvs_A = revenue_model.make_rvs(group='A', N_sample=10000)
rvs_B = revenue_model.make_rvs(group='B', N_sample=10000)
print(f'Expected revenue A: {rvs_A.mean():.2f}')
print(f'Expected revenue B: {rvs_B.mean():.2f}')

# Difference
rvs_diff = revenue_model.make_rvs_diff(N_sample=10000)
print(f'Revenue diff (A-B): {rvs_diff.mean():.2f}')

# Cumulative posterior
_, rvs_cum, _ = revenue_model.make_cum_posterior(group='A')
print(f'Cumulative steps: {len(rvs_cum)}')

# plot the final posterior for the composite model
fig = revenue_model.plot_final_posterior()
plt.show()

# get Bayesian statistics for the composite model
stats = revenue_model.summary_result(rope_interval=[-5,  5])
print(stats)

revenue_model.plot_bayesian_metrics(rope_interval=[-5, 5])
plt.show()